<a href="https://colab.research.google.com/github/Shrinathrajeshirke/deep-learning-churn-prediction/blob/main/simpleRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### End to end Deep Learning project using RNN

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [2]:
## read the dataset
max_features = 10000 #vocabulary size
(X_train, y_train),(X_test, y_test) = imdb.load_data(num_words=max_features)

## shape train test data
print(f'training data shape: {X_train.shape}, training labels shape: {y_train.shape}')
print(f'testing data shape: {X_test.shape}, testing labels shape: {y_test.shape}')

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
training data shape: (25000,), training labels shape: (25000,)
testing data shape: (25000,), testing labels shape: (25000,)


In [5]:
## inspect sample review and its label
print('X_train \n',X_train[0])
print('y_train \n',y_train[0])

X_train 
 [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
y_train 
 1


In [8]:
## sample review
sample_review = X_train[0]
sample_label = y_train[0]

### mapping of words index back to words
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}
decoded_review = ' '.join([reverse_word_index.get(i-3,'?') for i in sample_review])
decoded_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [9]:
## prepadding

max_len = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

In [10]:
## train simple RNN

model = Sequential()
model.add(Embedding(max_features, 128, input_length=max_len))
model.add(SimpleRNN(128,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
## create an instance of earlystoping callback
from tensorflow.keras.callbacks import EarlyStopping
EarlyStopping(monitor='val_loss', patience = 5, restore_best_weights=True)

In [16]:
### train the model

model.fit(
    X_train, y_train, epochs=10,batch_size=32,
    validation_split=0.2,
    validation_data=(X_test, y_test),
)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 196s 247ms/step - accuracy: 0.5747 - loss: 1521.4957 - val_accuracy: 0.6279 - val_loss: 0.6312
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 209s 257ms/step - accuracy: 0.5598 - loss: nan - val_accuracy: 0.5000 - val_loss: nan
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 201s 257ms/step - accuracy: 0.5032 - loss: nan - val_accuracy: 0.5000 - val_loss: nan
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 192s 244ms/step - accuracy: 0.4968 - loss: nan - val_accuracy: 0.5000 - val_loss: nan
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 200s 256ms/step - accuracy: 0.5028 - loss: nan - val_accuracy: 0.5000 - val_loss: nan
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 193s 244ms/step - accuracy: 0.5011 - loss: nan - val_accuracy: 0.5000 - val_loss: nan
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 191s 245ms/step - accuracy: 0.5018 - loss: nan - val_accuracy: 0.5000 - val_loss: nan
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 201s 244ms/step - accuracy: 0.4979 - loss: nan - val_accuracy: 0.5000

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
# Define a permanent path
permanent_path = '/content/drive/MyDrive/Colab Notebooks/simple_rnn.keras'

# Save the model to the permanent location
model.save(permanent_path)